In [101]:
import astroalign as aa
import numpy as np
import pandas as pd

from pathlib import Path
from trout.files.reference_log_file import ReferenceLogFile

In [247]:
ref_71 = ReferenceLogFile.get_ref_revised_71()

In [226]:
our_stars_to_webda_mapping = {
    (552, 399) : (20.84, -4.56), # 71
    (540, 427) : (16.46, -1.79), # 12
    (541, 454) : (13.95, 2.20), # 86
    (542, 476) : (11.87, 5.43), # 67
    (548, 494) : (10.97, 8.69), # 46
    
    #1 => WEBDA #2
    (746.58, 459.64) : (42.485, 23.800) #1# 2
}

In [227]:
our_stars, webda_stars = zip(*our_stars_to_webda_mapping.items())

In [228]:
# Find transformation matrix
t, _ = aa.find_transform(
    source=our_stars, 
    target=webda_stars)

In [229]:
t.scale, t.rotation, t.translation

(0.17411175849693364,
 0.6160826129483823,
 array([ -17.30499281, -116.72948101]))

In [230]:
# t matrix transforms to webda coordinates
# we use the inverse matrix to transfer to 
# our coordinates
inverse_mat = t.inverse
inverse_mat.scale, inverse_mat.rotation, inverse_mat.translation

(5.743437483101473, -0.6160826129483822, array([468.51912432, 489.73724913]))

In [262]:
def convert_from_ref71_coordinate_to_webda(position):
    return t(position)

In [263]:
def convert_from_webda_to_our_coordinates(position):
    return inverse_mat(position)

In [233]:
webda_xy_file = Path('webda_xy.txt')
_webda_df = pd.read_csv(webda_xy_file, delim_whitespace=True)

class Webda:
    stars = [star_no for star_no in _webda_df['No']]
    
    @classmethod
    def get_star(cls, star_no):
        return _webda_df[df['No'] == star_no]
    
    @classmethod
    def get_star_xy(cls, star_no):
        star_data = cls.get_star(star_no)
        return star_data.X.values[0], star_data.Y.values[0]
    
    @classmethod
    def get_star_mag(cls, star_no):
        star_data = cls.get_star(star_no)
        return star_data.Mag.values[0]

In [234]:
def distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [248]:
def find_star_by_coordinate(x, y, stars, get_star_xy, threshold=1):
    # Returns the star number at given X, Y coordinate
    # If more than 1 star are present, returns closest
    # If none are present returns none
    
    # Potential star is a iterable of starno, distance
    # We store the distance so that we can later identify
    # the closest star
    potential_stars = []
    for star in stars:
        dist = distance([x, y], get_star_xy(star))
        if dist <= threshold:
            potential_stars.append((star, dist))
    if len(potential_stars) > 0:
        return sorted(potential_stars, key=lambda x: x[1])[0][0]
    
def find_webda_star_by_coordinate(x, y, threshold=1):
    return find_star_by_coordinate(x, y, Webda.stars, Webda.get_star_xy, threshold)

def find_ref71_star_by_coordinate(x, y, threshold=1):
    return find_star_by_coordinate(x, 
                                   y, 
                                   range(1, len(ref_71.data()) + 1), 
                                   ref_71.get_star_xy, threshold)

In [238]:
# Checks webda star locater
assert find_webda_star_by_coordinate(-78, 72.8) == 1
assert find_webda_star_by_coordinate(33.5, 13.5) == 112
assert find_webda_star_by_coordinate(12, 15) == None

In [249]:
# Checks ref71 star locater
trials = 5
import random

for trial in range(trials):
    star = random.randint(1, 2500)
    x, y = ref_71.get_star_xy(star)
    new_x, new_y = x + random.random(),  y + random.random()
    if distance([new_x, new_y], [x, y]) < 1:
        assert find_ref71_star_by_coordinate(new_x, new_y) == star
    else:
        assert find_ref71_star_by_coordinate(new_x, new_y) == None

In [74]:
# Dictionary of our star numbers mapped to webda star position
our_stars_webda_position = {}
for star in range(1, 2508):
    our_stars_webda_position[star] = t(rev_71.get_star_xy(star))

def get_webda_coordinates(star_no):
    data = our_stars_webda_position.get(star_no)
    if data is not None:
        return data[0]

In [250]:
# Print all stars in webDA with our X, Y coordinates and star numbers

In [265]:
print(
    f"{'WebDA_#':<10s}"
    f"{'WebDA_X':<10s}"
    f"{'WebDA_Y':<10s}"
    f"{'ref71_X':<10s}"
    f"{'ref71_Y':<10s}"
    f"{'ref71_#':<10s}"
)
for webda_star in Webda.stars:
    webda_position = Webda.get_star_xy(webda_star)
    ref_revised_xy = convert_from_webda_to_our_coordinates(webda_position)[0]
    ref_star_number = find_ref71_star_by_coordinate(*ref_revised_xy, threshold=2) or ''
    print(
        f"{webda_star:<10}"
        f"{webda_position[0]:<10.2f}"
        f"{webda_position[1]:<10.2f}"
        f"{ref_revised_xy[0]:<10.2f}"
        f"{ref_revised_xy[1]:<10.2f}"
        f"{str(ref_star_number):<10s}"
    )

WebDA_#   WebDA_X   WebDA_Y   ref71_X   ref71_Y   ref71_#   
1         -78.22    73.75     346.62    1095.04             
2         42.48     23.80     746.63    460.32    1         
3         -57.09    -12.91    158.06    618.69              
4         3.75      20.07     552.71    571.37              
5         -6.04     -20.91    370.81    411.77              
6         -4.03     4.46      464.43    524.02              
7         -6.45     13.07     481.66    572.41              
8         -26.90    -0.76     339.90    575.45              
9         9.05      21.48     582.23    560.39    17        
10        13.93     51.13     703.51    683.18              
11        -27.33    33.63     452.02    738.08              
12        16.46     -1.79     539.73    426.72    25        
13        41.92     59.13     861.26    627.78              
14        -9.59     22.87     499.47    628.77              
15        53.71     61.26     923.59    598.64              
16        33.69     37.5

136       -10.93    -15.05    367.34    455.46              
137       -24.71    19.68     418.01    663.99              
138       -5.86     80.50     708.21    886.53              
139       -4.24     73.25     691.75    847.17              
140       -1.11     64.89     678.67    797.59              
141       4.12      62.01     693.63    766.74              
142       4.13      -21.79    415.56    373.89              
143       19.00     63.12     767.06    722.55              
144       30.51     61.59     815.94    677.18              
145       43.28     56.73     859.67    612.02              
146       48.81     60.05     896.61    609.23              
147       44.29     63.47     886.77    640.26              
148       54.57     67.42     948.07    624.66              
149       59.14     59.59     943.51    572.79              
150       74.01     24.15     895.59    357.32              
151       65.16     50.48     941.49    510.11              
152       57.58     52.7

1152      0.85      -101.06   137.11    13.20               
1153      -1.55     -91.23    158.48    67.24               
1154      -4.45     -91.62    143.59    75.04               
1155      1.42      -48.28    314.94    258.71              
1165      0.00      0.00      468.52    489.74    243       
1175      -4.99     42.15     585.02    703.88              
1188      -15.21    -10.53    362.28    490.86              
1193      -9.37     -49.29    261.01    289.79              
1194      -16.15    -67.07    170.22    228.95              
1201      -24.58    -23.50    275.31    461.16              
1208      -20.03    33.28     485.08    712.21              
1227      -42.17    -15.74    218.61    555.91              
1230      -45.48    -8.27     227.89    601.91              
1232      -40.47    14.63     327.37    692.63              
1233      -35.35    16.87     358.80    686.13              
1240      -53.60    58.36     410.95    941.19              
1243      -47.97    5.09

In [269]:
print(
    f"{'ref71_#':<10s}"
    f"{'ref71_X':<10s}"
    f"{'ref71_Y':<10s}"
    f"{'WebDA_#':<10s}"
    f"{'WebDA_X':<10s}"
    f"{'WebDA_Y':<10s}"
)
for ref_revised_star in range(1, len(ref_71.data()) + 1):
    ref_revised_xy = ref_71.get_star_xy(ref_revised_star)
    webda_position = convert_from_ref71_coordinate_to_webda(ref_revised_xy)[0]
    webda_star_no = find_webda_star_by_coordinate(*webda_position, threshold=1) or ''
    print(
        f"{ref_revised_star:<10}"
        f"{ref_revised_xy[0]:<10.2f}"
        f"{ref_revised_xy[1]:<10.2f}"
        f"{webda_position[0]:<10.2f}"
        f"{webda_position[1]:<10.2f}"
        f"{str(webda_star_no):<10s}"
    )

ref71_#   ref71_X   ref71_Y   WebDA_#   WebDA_X   WebDA_Y   
1         746.58    459.64    42.54     23.70     2         
2         547.57    493.95    10.81     8.55      46        
3         462.38    523.01    -4.22     4.11      6         
4         230.97    347.20    -19.42    -44.15    49        
5         112.45    420.52    -43.63    -45.66              
6         153.63    614.49    -57.30    -13.95              
7         957.99    389.36    79.65     34.98     55        
8         500.73    376.04    16.02     -12.92    47        
9         550.72    571.94    3.41      19.95     4         
10        369.82    409.03    -5.91     -21.40    5         
11        580.00    819.65    -17.35    58.10     52        
12        479.33    572.02    -6.74     12.78     7         
13        336.75    573.45    -27.15    -1.36     8         
14        586.45    96.41     56.33     -44.03    51        
15        366.07    362.05    -1.71     -28.45    83        
16        700.99    686.

135       73.33     119.07    -18.86    -92.43              
136       105.60    862.38    -89.06    16.44               
137       495.40    672.85    -14.60    28.72     110       
138       456.43    301.43    17.23     -27.97    109       
139       18.93     839.16    -99.04    4.42                
140       217.43    645.43    -51.34    -3.14     1246      
141       432.50    809.94    -37.33    41.88     106       
142       796.93    135.77    82.28     -17.26    1021      
143       913.26    550.13    57.12     53.33     152       
144       46.61     647.05    -75.78    -20.09              
145       528.60    630.03    -5.58     25.98     103       
146       671.97    444.90    33.42     14.10     112       
147       675.02    392.34    39.14     6.94      107       
148       219.84    53.11     8.59      -87.06              
149       185.68    624.47    -53.75    -9.31               
150       234.86    492.93    -33.52    -23.05    105       
151       185.69    624.

271       306.72    735.79    -47.75    18.69     179       
272       70.17     480.44    -55.67    -41.40              
273       334.74    940.86    -64.40    50.65               
274       334.75    940.80    -64.39    50.64               
275       347.42    34.01     28.64     -76.94              
276       431.89    133.42    30.64     -54.32    1104      
277       316.13    742.67    -47.10    20.61     180       
278       22.02     177.65    -32.05    -89.27              
279       438.54    545.53    -9.87     4.91      131       
280       399.03    186.95    20.59     -50.02    85        
281       203.93    408.89    -29.46    -38.11              
282       885.53    645.09    43.63     64.03     147       
283       112.49    812.48    -83.06    10.04               
284       942.94    343.50    82.13     26.95               
285       685.09    468.72    32.89     18.80               
286       895.96    850.93    24.40     94.33               
287       911.10    154.

407       482.61    693.62    -18.51    30.39               
408       877.40    73.55     99.97     -18.00              
409       666.28    518.88    25.17     24.04               
410       105.55    732.96    -76.05    -1.96               
411       677.54    379.40    40.80     5.35                
412       105.34    733.24    -76.11    -1.94               
413       797.50    90.61     86.90     -23.62              
414       697.05    219.73    59.64     -15.38              
415       93.96     79.64     -11.97    -95.96              
416       207.95    858.33    -74.11    26.16               
417       839.07    103.62    91.50     -17.59              
418       819.18    939.56    4.57      99.20               
419       47.19     36.48     -14.27    -106.80             
420       477.05    730.27    -22.99    35.04               
421       459.96    170.57    30.89     -46.22              
422       819.41    939.46    4.62      99.21               
423       198.12    521.

543       270.11    767.64    -56.15    19.53               
544       706.53    93.59     73.68     -32.35              
545       622.54    847.47    -14.10    66.33               
546       199.30    163.82    -5.47     -73.40              
547       450.08    420.01    4.39      -11.76              
548       818.27    739.05    24.62     70.62               
549       481.29    822.14    -31.63    48.52               
550       302.47    469.59    -21.57    -19.57              
551       481.12    822.27    -31.67    48.52               
552       616.43    933.09    -23.59    77.88               
553       59.01     264.53    -35.53    -73.20              
554       481.10    822.40    -31.68    48.54               
555       297.54    319.58    -7.18     -41.38              
556       761.44    368.44    53.83     12.23               
557       964.72    913.84    27.84     110.19              
558       507.59    426.15    11.95     -5.10     37        
559       341.08    291.

...

1902      582.86    727.92    -7.72     45.35               
1903      865.49    458.97    59.51     35.57               
1904      919.86    676.48    45.35     71.95               
1905      959.03    1005.02   17.86     122.57              
1906      850.09    527.13    50.46     43.70               
1907      275.62    602.07    -38.71    -3.44               
1908      111.58    500.95    -51.85    -34.32              
1909      44.64     120.97    -23.13    -95.05              
1910      889.98    793.95    29.28     85.63               
1911      215.08    567.65    -43.85    -14.43              
1912      814.95    948.13    3.11      99.99               
1913      799.11    53.35     90.88     -28.75              
1914      460.89    963.08    -48.71    66.49               
1915      122.73    154.96    -15.46    -82.36              
1916      818.90    17.54     97.30     -31.85              
1917      473.98    290.39    20.83     -27.78              
1918      122.73    154.

2038      992.02    228.06    100.72    15.48               
2039      122.07    773.04    -77.73    5.40                
2040      841.89    902.21    11.56     96.18               
2041      785.91    410.06    53.12     20.61               
2042      472.06    440.94    5.41      -6.58               
2043      829.70    474.80    52.83     34.22               
2044      680.01    600.90    18.87     37.07               
2045      203.68    821.83    -71.05    20.55               
2046      554.32    342.08    27.05     -12.35              
2047      979.76    35.69     118.33    -13.09              
2048      384.12    113.40    25.87     -61.97              
2049      343.93    527.17    -21.47    -7.22               
2050      368.29    292.03    5.65      -38.18              
2051      866.43    195.93    86.10     -1.72               
2052      848.70    736.72    29.18     73.35               
2053      661.87    644.97    11.86     41.51               
2054      584.24    955.

2174      205.32    629.64    -51.48    -6.60               
2175      771.69    643.05    27.66     52.29               
2176      444.55    47.94     41.04     -65.19              
2177      685.06    1005.23   -21.09    95.04               
2178      515.02    797.94    -24.40    48.47               
2179      621.72    490.00    21.74     15.45               
2180      93.83     1006.20   -105.20   35.69               
2181      471.47    226.89    26.86     -37.05              
2182      446.17    936.30    -48.10    61.21               
2183      961.90    368.49    82.31     32.41               
2184      829.30    848.04    15.22     87.21               
2185      567.01    196.06    43.54     -31.82              
2186      897.96    408.89    69.16     31.72               
2187      610.29    411.06    28.06     3.08                
2188      928.12    113.56    103.16    -7.22               
2189      612.01    482.49    21.12     13.41               
2190      944.67    847.

2310      619.71    552.07    15.21     24.07               
2311      703.50    775.48    4.64      64.25               
2312      989.06    321.97    90.85     28.53               
2313      365.76    140.12    20.57     -60.02              
2314      53.49     702.00    -80.33    -11.59              
2315      787.00    962.49    -2.31     99.22               
2316      951.09    773.84    39.99     88.92               
2317      366.51    664.57    -32.09    14.58               
2318      154.80    42.94     0.37      -95.05              
2319      786.45    325.09    61.74     8.59                
2320      685.50    985.81    -19.08    92.32               
2321      514.03    569.02    -1.51     15.84               
2322      892.97    414.32    67.90     31.99               
2323      602.06    231.67    44.94     -23.24              
2324      979.68    967.12    24.61     119.26              
2325      378.86    817.08    -45.67    37.50               
2326      788.04    770.

2446      982.32    842.67    37.50     101.85              
2447      249.54    416.88    -23.79    -32.38              
2448      846.97    1008.28   1.61      111.76              
2449      947.19    815.32    35.26     94.42               
2450      940.21    914.93    24.25     107.88              
2451      966.33    966.21    22.80     117.79              
2452      947.50    960.04    20.75     115.02              
2453      196.88    102.62    0.35      -82.34              
2454      606.91    827.15    -14.28    61.87               
2455      40.69     229.55    -34.62    -80.02              
2456      796.45    981.59    -2.89     102.89              
2457      371.84    950.12    -60.06    55.69               
2458      867.96    987.09    6.72      110.86              
2459      431.03    1003.01   -56.97    69.16               
2460      213.33    435.99    -30.86    -33.31              
2461      778.02    893.70    3.34      88.54               
2462      909.97    981.